In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# step 0 -> prepare data
bc = datasets.load_breast_cancer() # for binary classification
x,y = bc.data, bc.target
n_samples, n_features = x.shape
print(n_samples, n_features)
x_train,x_test, y_train,y_test = train_test_split(x,y,test_size = 0.2)

# scale
# The transform method uses the parameters learned from the training data
# (mean and standard deviation) and applies the same transformation to the test data (x_test).
# It ensures that the test data is scaled in the same way as the training data, based on the previously learned parameters.
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# tranform data to tensors
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape y_train and y_test
y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)
print(x_train.size())
print(y_train.size())

569 30
torch.Size([455, 30])
torch.Size([455, 1])


In [3]:
# model
class logisticRegression(nn.Module):
    def __init__(self, num_input_features):
        super(logisticRegression, self).__init__()
        self.linear = nn.Linear(num_input_features,1)
        
    def forward(self,x):
        y_hat = torch.sigmoid(self.linear(x))
        return y_hat
    

model = logisticRegression(n_features)

In [4]:
# loss and optimisers
criterion = nn.BCELoss()
learning_rate = 0.1
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
# training loop
n_epochs = 100
for epoch in range(n_epochs):
    # forward pass
    y_hat = model(x_train)
    
    # calc loss
    loss = criterion(y_hat, y_train)
    
    # backward pass
    loss.backward()
    
    # update weights
    optimiser.step()
    
    # reload gradients to zero
    optimiser.zero_grad()
    
    if epoch % 10==0:
        print(f'epoch: {epoch}, loss: {loss.item():.4f}')
    

epoch: 0, loss: 0.7319
epoch: 10, loss: 0.2578
epoch: 20, loss: 0.1951
epoch: 30, loss: 0.1667
epoch: 40, loss: 0.1498
epoch: 50, loss: 0.1383
epoch: 60, loss: 0.1298
epoch: 70, loss: 0.1232
epoch: 80, loss: 0.1179
epoch: 90, loss: 0.1136


In [6]:
with torch.no_grad():
    y_hat_test = model(x_test)
    y_test_cls = y_hat_test.round()
    acc = y_test_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9912
